<a href="https://colab.research.google.com/github/vfuy/ML-000/blob/main/Week02/gt_mltc_ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [3]:
def target_mean_v1(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  for i in range(data.shape[0]):
    groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
    result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
  return result

In [4]:
def target_mean_v2(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data.loc[i, x_name] not in value_dict.keys():
      value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] = 1
    else:
      value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] += 1
  for i in range(data.shape[0]):
    result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
  return result

In [5]:
%%timeit
result_1 = target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 23.6 s per loop


In [6]:
%%timeit
result_2 = target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 274 ms per loop


In [7]:
result_1 = target_mean_v1(data, 'y', 'x')
result_2 = target_mean_v2(data, 'y', 'x')
diff = np.linalg.norm(result_1 - result_2)
print(diff)

0.0


In [8]:
%load_ext Cython

In [9]:
%%cython -a

import numpy as np
cimport numpy as cnp

def target_mean_v3(data, y_name, x_name):
  cdef:
    int length = data.shape[0]
    dict value_dict = {}
    dict count_dict = {}
    cnp.ndarray[cnp.float64_t] result = np.zeros(length, dtype=np.float64)
    cnp.ndarray[cnp.int_t] y = data[y_name].values
    cnp.ndarray[cnp.int_t] x = data[x_name].values
  for i in range(length):
    if x[i] not in value_dict:
      value_dict[x[i]] = y[i]
      count_dict[x[i]] = 1
    else:
      value_dict[x[i]] += y[i]
      count_dict[x[i]] += 1
  for i in range(length):
    result[i] = (value_dict[x[i]] - y[i]) / (count_dict[x[i]] - 1)
  return result

In [10]:
%%timeit
result_3 = target_mean_v3(data, 'y', 'x')

1000 loops, best of 3: 1.07 ms per loop


In [11]:
result_2 = target_mean_v2(data, 'y', 'x')
result_3 = target_mean_v3(data, 'y', 'x')
diff = np.linalg.norm(result_2 - result_3)
print(diff)

0.0
